In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
os.chdir("d:\\ML Projects\\house_price_prediction")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_file_path = r"D:\ML Projects\house_price_prediction\housing\artifact\data_ingestion\2022-07-12_12-26-23\ingested_data\train\housing.csv"

In [ ]:
df = pd.read_csv(train_file_path)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
y = df[["median_house_value"]]

In [ ]:
x = df.drop(columns="median_house_value",axis = 1)

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
x

In [ ]:
y

In [ ]:
x.isnull().sum()

In [ ]:
y.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
simple_imputer = SimpleImputer(strategy="median")

In [ ]:
numerical_features =x.drop(columns="ocean_proximity",axis=1)

In [ ]:
numerical_features.head(2)

In [ ]:
simple_imputer.fit_transform(numerical_features)

In [ ]:
df = simple_imputer.transform(numerical_features)

In [ ]:
df

In [ ]:
simple_imputer.feature_names_in_

In [ ]:
simple_imputer

In [ ]:
simple_imputer.statistics_

In [ ]:
x["longitude"].median()

In [ ]:
x["latitude"].median()

In [ ]:
x["ocean_proximity"].value_counts()

In [ ]:
simple_imputer_cat = SimpleImputer(strategy ="most_frequent")

In [ ]:
categorical_features = x[["ocean_proximity"]]

In [ ]:
simple_imputer_cat.fit_transform(categorical_features)

In [ ]:
simple_imputer_cat.transform(categorical_features)

In [ ]:
simple_imputer_cat.feature_names_in_

In [ ]:
simple_imputer_cat.statistics_

In [ ]:
test_file_path = r"D:\ML Projects\house_price_prediction\housing\artifact\data_ingestion\2022-07-12_12-26-23\ingested_data\test\housing.csv"

In [ ]:
new_df = pd.read_csv(test_file_path)

In [ ]:
new_df.head(3)

In [ ]:
X,Y = new_df.drop(columns="median_house_value"),new_df[["median_house_value"]]

In [ ]:
X_numeric = X.drop(columns ="ocean_proximity")

In [ ]:
simple_imputer.fit_transform(X_numeric)

In [ ]:
new_df = simple_imputer.transform(X_numeric)

In [ ]:
new_df

In [ ]:
simple_imputer.feature_names_in_

In [ ]:
simple_imputer.statistics_

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
from housing.config.configuration import Configuration
config = Configuration()

In [ ]:
data_ingestion_config = config.get_data_ingestion_config()

In [ ]:
data_validation_config = config.get_data_validation_config()

In [ ]:
from housing.component.data_ingestion import DataIngestion
data_ingestion = DataIngestion(data_ingestion_config)

In [ ]:
data_ingestion_artifact = data_ingestion.initiate_data_ingestion()

In [ ]:
from housing.component.data_validation import DataValidation
data_validation = DataValidation(data_ingestion_config,data_ingestion_artifact)

In [ ]:
data_transformation_config = config.get_data_transformation_config()

In [ ]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import pandas as pd
from housing.util.util import read_yaml_file
from housing.constant import *
from housing.logger import logging
from housing.exception import ExceptionHendler


In [ ]:
df.columns

In [ ]:
numerical_columns = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',]

In [ ]:
categorical_columns = ['ocean_proximity']

In [ ]:
data_transformation_config.add_bedroom_per_room

In [ ]:
COLUMN_TOTAL_ROOMS = "total_rooms"
COLUMN_POPULATION = "population"
COLUMN_HOUSEHOLDS = "households"
COLUMN_TOTAL_BEDROOM = "total_bedrooms"

class FeatureGenerator(BaseEstimator, TransformerMixin):

    def __init__(self, add_bedrooms_per_room=True,
                 total_rooms_ix=3,
                 population_ix=5,
                 households_ix=6,
                 total_bedrooms_ix=4, columns=None):
        """
        FeatureGenerator Initialization
        add_bedrooms_per_room: bool
        total_rooms_ix: int index number of total rooms columns
        population_ix: int index number of total population columns
        households_ix: int index number of  households columns
        total_bedrooms_ix: int index number of bedrooms columns
        """
        try:
            self.columns = columns
            if self.columns is not None:
                total_rooms_ix = self.columns.index(COLUMN_TOTAL_ROOMS)
                population_ix = self.columns.index(COLUMN_POPULATION)
                households_ix = self.columns.index(COLUMN_HOUSEHOLDS)
                total_bedrooms_ix = self.columns.index(COLUMN_TOTAL_BEDROOM)

            self.add_bedrooms_per_room = add_bedrooms_per_room
            self.total_rooms_ix = total_rooms_ix
            self.population_ix = population_ix
            self.households_ix = households_ix
            self.total_bedrooms_ix = total_bedrooms_ix
        except Exception as e:
            raise e

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        try:
            room_per_household = X[:, self.total_rooms_ix] / \
                                 X[:, self.households_ix]
            population_per_household = X[:, self.population_ix] / \
                                       X[:, self.households_ix]
            if self.add_bedrooms_per_room:
                bedrooms_per_room = X[:, self.total_bedrooms_ix] / \
                                    X[:, self.total_rooms_ix]
                generated_feature = np.c_[
                    X, room_per_household, population_per_household, bedrooms_per_room]  ## Here _c is Concatination symbol
            else:
                generated_feature = np.c_[
                    X, room_per_household, population_per_household]

            return generated_feature
        except Exception as e:
            raise ExceptionHendler(e,sys) from e


In [ ]:
num_pipeline = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy="median")),
                ('feature_generator', FeatureGenerator(
                    add_bedrooms_per_room=data_transformation_config.add_bedroom_per_room,
                    columns=numerical_columns
                )),
                ('scaler', StandardScaler())
            ]
            )

cat_pipeline = Pipeline(steps=[
                 ('impute', SimpleImputer(strategy="most_frequent")),
                 ('one_hot_encoder', OneHotEncoder()),
                 ('scaler', StandardScaler(with_mean=False))
            ]
            )

logging.info(f"Categorical columns: {categorical_columns}")
logging.info(f"Numerical columns: {numerical_columns}")


preprocessing = ColumnTransformer([
                ('num_pipeline', num_pipeline, numerical_columns),
                ('cat_pipeline', cat_pipeline, categorical_columns),
            ])

In [ ]:
preprocessing

In [ ]:
new_df

In [ ]:
train_file_path = data_ingestion_artifact.test_file_path

In [ ]:
train_df = pd.read_csv(train_file_path)

In [ ]:
preprocessing.fit(train_df)

In [ ]:
preprocessing.transform(train_df)

In [ ]:
preprocessing.fit_transform(train_df)

In [ ]:
preprocessing.feature_names_in_

In [ ]:
preprocessing.get_feature_names_out

In [ ]:
preprocessing.get_params

In [ ]:
preprocessing.output_indices_

In [ ]:
preprocessing._validate_transformers

In [ ]:
numerical_features

In [ ]:
X_numeric.head()

In [ ]:
from sklearn.base import BaseEstimator,TransformerMixin

In [ ]:
class Feature_Generator(BaseEstimator, TransformerMixin):
    def __init__(self,strategy ="median"):
        try:
            self.strategy =strategy
        except Exception as e:
           print(e)
    
    def fit(self,X):
        self.features = X.columns
        self.statistics_ = []

        for column in X.columns:
            self.statistics_.append(X[column].median())

        return self

    def transform(self,X:pd.DataFrame):
        for idx,column in enumerate(X.columns):
            X[column].fillna(self.statistics_[idx])
        return X

    #def fit_transform(self,X):
        #self.fit(X)
        #return self.fit_transform(X)


    

In [ ]:
response = Feature_Generator().fit(numerical_features)

In [ ]:
response.transform(X_numeric)

In [ ]:
response.fit_transform(X_numeric)

In [ ]:
X_numeric.isna().sum()

In [ ]:
test_df_a = preprocessing.fit_transform(new_df)

In [ ]:
test_df = pd.DataFrame(test_df_a)

In [ ]:
test_df

In [ ]:
test_df.columns = new_df.columns

In [ ]:
response

In [ ]:
gen.transform(X_numeric)

In [ ]:
gen.statistics_

In [ ]:
X_numeric.isna().sum()

In [ ]:
gen.fit_transform(X_numeric)

In [ ]:
gen.transform(X_numeric)

In [ ]:
X_numeric.isnull().sum()

In [ ]:
X_numeric

In [ ]:
X_numeric

In [ ]:
train_file_path

In [ ]:
data = pd.read_csv(train_file_path)

In [ ]:
data.head(3)

In [ ]:
data.columns

In [ ]:
stats

In [ ]:
for idx,column in enumerate(X_numeric.columns):
    print(idx,column)

In [ ]:
stats = [-118.46, 34.22, 28.0, 2159.5, 442.0, 1172.5, 416.0, 3.5132000000000003]
for idx,column in enumerate(X_numeric.columns):
    X_numeric[column].fillna(stats[idx])

    #print(X_numeric.isnull().sum())

In [ ]:
X_numeric.isna().sum()

In [ ]:
X_numeric

In [ ]:
stats = []
for column in X_numeric.columns:
    stats.append(X_numeric[column].median())
print(stats)

In [ ]:
stats

In [ ]:
class test(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass

In [ ]:
obj=test()

In [ ]:
test_file_path = data_ingestion_artifact.test_file_path

In [ ]:
test_df = pd.read_csv(test_file_path)

In [ ]:
obj.fit(test_df)

In [ ]:
os.getcwd()

In [ ]:
os.chdir("d:\\ML Projects\\house_price_prediction")

In [ ]:
from housing.config.configuration import Configuration

In [ ]:
config = Configuration()

In [ ]:
data_ingestion_config = config.get_data_ingestion_config()

In [ ]:
data_validation_config = config.get_data_validation_config()

In [ ]:
data_validation_config.report_file_path

In [ ]:
data_validation_config.report_page_file_path

In [ ]:
from housing.component.data_ingestion import DataIngestion
data_ingestion = DataIngestion(data_ingestion_config)

In [ ]:
data_ingestion_artifact = data_ingestion.initiate_data_ingestion()

In [ ]:
from housing.component.data_validation import DataValidation

In [ ]:
data_validation = DataValidation(data_validation_config,data_ingestion_artifact)

In [ ]:
data_validation_artifact = data_validation.initiate_data_validation()

In [ ]:
data_transformation_config = config.get_data_transformation_config()

In [ ]:
from housing.component.data_transformation import DataTransformation

In [ ]:
data_transformation = DataTransformation(config=config,data_transformation_config=data_transformation_config,data_ingestion_artifact=data_ingestion_artifact,
data_validation_artifact=data_validation_artifact)

In [ ]:
data_transformation.get_data_transformer_object()

In [1]:
os.getcwd()

'd:\\ML Projects\\house_price_prediction\\notebook'

In [2]:
os.chdir("d:\\ML Projects\\house_price_prediction")

In [3]:
from housing.config.configuration import Configuration

In [4]:
config = Configuration()

In [5]:
data_ingestion_config=config.get_data_ingestion_config()
data_validation_config = config.get_data_validation_config()
data_transformation_config = config.get_data_transformation_config()

In [6]:
from housing.component.data_ingestion import DataIngestion

In [7]:
data_ingestion = DataIngestion(data_ingestion_config)

In [8]:
data_ingestion_artifact = data_ingestion.initiate_data_ingestion()

In [9]:
data_ingestion_artifact

DataIngestionArtifact(train_file_path='d:\\ML Projects\\house_price_prediction\\housing\\artifact\\data_ingestion\\2022-07-20_20-25-39\\ingested_data\\train\\housing.csv', test_file_path='d:\\ML Projects\\house_price_prediction\\housing\\artifact\\data_ingestion\\2022-07-20_20-25-39\\ingested_data\\test\\housing.csv', is_ingested=True, message='Data Ingestion completed successfully')

In [10]:
from housing.component.data_validation import DataValidation

In [11]:
data_validation = DataValidation(data_validation_config,data_ingestion_artifact)

In [12]:
data_validation_artifact = data_validation.initiate_data_validation()

Total Number Of Columns = [10]
            Columns Names are : [Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')],
            Numerical Feature Columns :[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']],
            Categorical Features Column : [['ocean_proximity']],
            Values in Ocean Proximity Column : [<1H OCEAN     7277
INLAND        5262
NEAR OCEAN    2124
NEAR BAY      1847
ISLAND           2
Name: ocean_proximity, dtype: int64]
            Validation Status =[True]
            


In [13]:
train_df,test_df=data_validation.get_train_df_and_test_df()

In [14]:
from housing.constant import *

In [15]:
from housing.util.util import read_yaml_file

In [16]:
from housing.component.data_transformation import DataTransformation

In [17]:
data_transformation = DataTransformation(config,data_transformation_config= data_transformation_config,data_ingestion_artifact= data_validation_artifact,data_validation_artifact=data_validation_artifact)

In [18]:
transformer_obj = data_transformation.get_data_transformer_object()

In [19]:
train_arr = transformer_obj.fit_transform(train_df)

In [20]:
train_arr

array([[-60.67637885,  18.01769717,   2.30626612, ...,   0.        ,
          0.        ,   0.        ],
       [-58.56324628,  15.4778193 ,   0.55668493, ...,   0.        ,
          0.        ,   2.9869105 ],
       [-59.46744721,  16.54428736,   3.4991624 , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [-61.3058226 ,  17.98027724,   3.81726807, ...,   0.        ,
          0.        ,   0.        ],
       [-61.29583143,  17.91946985,   1.11336985, ...,   0.        ,
          0.        ,   0.        ],
       [-61.01607865,  18.69593344,   2.14721329, ...,   0.        ,
          0.        ,   0.        ]])

In [21]:
train_arr.shape

(16512, 16)

In [22]:
test_arr = transformer_obj.fit_transform(test_df)

In [23]:
test_arr.shape

(4128, 16)

In [24]:
data_transformation.initiate_data_transformation()

ExceptionHendler: error occure in file : [d:\ML Projects\house_price_prediction\housing\component\data_transformation.py]
        at Try block line Number : [154] and
        Exception_block_line_number : [234]
          Error_message is : ['DataValidationArtifact' object has no attribute 'train_file_path']        
        